**Testing the Model**

1. Importing Required Libraries.

In [ ]:
import os
import random
from PIL import Image, ImageDraw
from torchvision.transforms import ToPILImage

2. TestImageDataset, is a custom class of a PyTorch dataset designed to handle test images without annotations.
    - Loads test images from the specified directory.

In [ ]:
class TestImageDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.image_paths = sorted([os.path.join(root, fname) for fname in os.listdir(root) if fname.endswith(('.jpg', '.png'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image_path

3. Load the Test Dataset:
    - Select a Random Subset of Test Images (20).

In [ ]:

print("Loading test dataset...")
test_dataset = TestImageDataset(
    root='/kaggle/input/coco-2017-dataset/coco2017/test2017',
    transform=transform
)

random.seed(42)
test_indices = random.sample(range(len(test_dataset)), 20)
test_subset = torch.utils.data.Subset(test_dataset, test_indices)

test_loader = DataLoader(test_subset, batch_size=1, shuffle=False)

4. Displaying detection.
    - Draw the predicted bounding boxes and labels on the input image.
    - Filter Low-Confidence Predictions (draw bounding box on predictions above 50 percent confidence score)
    - Add text label of class or category name.

In [ ]:
def draw_predictions(image, predictions, category_names, threshold=0.5):
    draw = ImageDraw.Draw(image)
    for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
        if score > threshold:
            box = box.tolist()
            label_text = category_names[label.item()]
            draw.rectangle(box, outline="red", width=2)
            draw.text((box[0], box[1]), f"{label_text} {score:.2f}", fill="yellow")
    return image

def filter_predictions(outputs, threshold=0.5):
    predictions = []
    for output in outputs:
        keep = output['scores'] > threshold
        predictions.append({
            'boxes': output['boxes'][keep],
            'labels': output['labels'][keep],
            'scores': output['scores'][keep],
        })
    return predictions

5. Helper functions to fix data:
    - Reverting the normalization applied during preprocessing so that the image can be visualized correctly.
    - Fixes invalid or inconsistent bounding box coordinates.

In [ ]:
def denormalize(tensor, mean, std):
    mean = torch.tensor(mean).view(3, 1, 1)
    std = torch.tensor(std).view(3, 1, 1)
    return tensor * std + mean

os.makedirs("/kaggle/working/test_results", exist_ok=True)

def correct_bounding_box(box):
    x0, y0, x1, y1 = box
    x0, x1 = min(x0, x1), max(x0, x1)
    y0, y1 = min(y0, y1), max(y0, y1)
    return [x0, y0, x1, y1]

6. Starting the Testing Process:
   - Removing the batch dimension from the image tensor.
   - Applying softmax to convert raw classification logits into probabilities.
   - Filters predictions based on a confidence threshold greater than 50 percent.
   - Reverting normalization applied during preprocessing to restore the image to its original color range.
   - Converting into PIL image for visualization.
   - Save the output image with bounding boxes.

   

In [ ]:

print("Starting testing...")
model.eval()

with torch.no_grad():
    for i, (image_tensor, image_path) in enumerate(test_loader):
       
        image_tensor = image_tensor.squeeze(0).to(device)
        
        image_tensor = image_tensor.unsqueeze(0)

        outputs = model(image_tensor)
        class_logits, bbox_regressions = outputs

        class_scores = torch.softmax(class_logits, dim=-1)
        top_scores, top_labels = torch.max(class_scores[0], dim=-1)
        filtered_boxes = bbox_regressions[0]

        keep = top_scores > 0.5
        top_scores = top_scores[keep]
        top_labels = top_labels[keep]
        filtered_boxes = filtered_boxes[keep]

        denormalized_image = denormalize(image_tensor.squeeze(0).cpu(), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]).clamp(0, 1)
        pil_image = transforms.ToPILImage()(denormalized_image)

        draw = ImageDraw.Draw(pil_image)
        for box, label, score in zip(filtered_boxes, top_labels, top_scores):
            box = correct_bounding_box(box.cpu().tolist())
            label_text = COCO_INSTANCE_CATEGORY_NAMES[label.item()]
            score_text = f"{score.item() * 100:.1f}%"
            draw.rectangle(box, outline="red", width=2)
            draw.text((box[0], box[1]), f"{label_text}: {score_text}", fill="yellow")

        output_path = f"/kaggle/working/test_results/test_image_{i + 1}.jpg"
        pil_image.save(output_path)

        print(f"Processed and saved: {output_path}")